# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [0]:
!wget -O peace.txt http://www.gutenberg.org/files/2600/2600-0.txt

--2019-10-07 19:59:15--  http://www.gutenberg.org/files/2600/2600-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3359549 (3.2M) [text/plain]
Saving to: ‘peace.txt’

peace.txt           100%[===================>]   3.20M  2.35MB/s    in 1.4s    

2019-10-07 19:59:17 (2.35 MB/s) - ‘peace.txt’ saved [3359549/3359549]



In [0]:
text = open('peace.txt', 'r').read()[2:]
len(text)

3227579

In [0]:
import string
import re

def preprocess_text(text):
    
    text = text.lower()
    
    punctuation = string.punctuation.replace('.', '') + '—“”‘’'
    translator = text.maketrans(punctuation, len(punctuation) * ' ')
    text = text.translate(translator)

    text = re.sub('\s+', ' ', text)

    return text

text = preprocess_text(text)
assert len(text) == 3141169

In [0]:
text = text.split('.')
text = [x.strip() for x in text]

In [0]:
from collections import Counter
from nltk import bigrams
from sklearn.base import TransformerMixin
import itertools
from copy import deepcopy



class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # tokenize text by symbols and fill in self.itos and self.stoi
        text_copy = deepcopy(" ".join(text))
        self.itos = list(set(text_copy))
        self.stoi = {element : index for index,element in enumerate(self.itos)}
        
        text = [[self.stoi[char] for char in element]for element in text]   
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            elements = list(itertools.chain.from_iterable(text_copy))
            bigrams_ = Counter(list(bigrams(elements)))
            new_token = ''.join(list(bigrams_.most_common(1)[0][0]))
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            # find occurences of the new_token in the text and replace them with new_id
            for index, element in enumerate(text):
              count = 0
              while count < len(element) -1:
                temp = element[count] + element[count+1]
                if temp == new_token:
                  text[index][count] = new_id
                  del text[index][count+1]
                count += 1
            
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        text = [[self.stoi[char] for char in element]for element in text]
        for token_id, token in enumerate(self.itos):
            for index, element in enumerate(text):
              count = 0
              while count < len(element) -1:
                temp = element[count] + element[count+1]
                if temp == token:
                  text[index][count] = token_id
                  del text[index][count+1]
                count += 1      
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        if isinstance(tok, int):
          result = self.itos[tok] 
        else:
          result = [self.itos[i] for i in tok]
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [0]:
import numpy as np
from nltk import trigrams
    
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = np.zeros((self.vocab_size, self.vocab_size, self.vocab_size))
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = np.array([self.get_proba(a,b, i, tau) for i in range(self.vocab_size)])
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        
        result = (self.proba[(a,b,c)] + self.delta) ** (1/tau) / sum (
            [(self.proba[(a,b,x)] + self.delta)**(1/tau) for x in 
             range(self.vocab_size)])
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        
        temp = Counter()
        for element in text:
         line = [start_token] + element + [end_token]
         temp.update(nltk.trigrams(line))
        self.proba = temp        
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    
    beam = [(input_seq,0)]
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam:
            if snt[-1] == end_token:
              candidates.append(snt)
              candidates_proba.append(snt_proba)
            else:    
                proba = lm.infer(snt[-2],snt[-1], tau)
                best_k = proba.argsort()[::-1][:k]               
            
                candidates.extend([snt+[int(element)] for element in best_k])
                candidates_proba.extend([snt_proba + el for el in np.log(proba[best_k])])
                
        beam = []
        for i in np.argsort(candidates_proba)[::-1][:k]:
          beam.append((candidates[i], candidates_proba[i]))
    return beam
    

In [157]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for i in range(10):
  print(bpe.decode(result[i][0]), result[i][1])

horse and the an -1.0048334157699486
horse the and th -2.151178622063114
horse and the th -2.1511786220631146
horse and the sa -3.333043490162556
horse and the so -3.550846727657047
horse said the a -3.586426454965451
horse the the an -3.776751794277742
horse and the sh -4.021718350853669
horse she and th -4.022227530008814
horse so the and -4.038659205655186


In [134]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for i in range(10):
  print(bpe.decode(result[i][0]), result[i][1])

her the and t -0.8758370811243786
her and the a -1.7326744147414819
her the the a -2.501383655829784
her and the t -2.8790462185438708
her and the s -3.138039438949162
her the said  -3.4420508801429968
her the the t -3.6477554596321724
her the the s -3.906748680037464
her the so th -3.9096885738256137
her the she a -4.372432563775484


In [135]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities
for i in range(10):
  print(bpe.decode(result[i][0]), result[i][1])

what of the th -9.202428511754622
what the and t -9.273372192252266
what the the a -9.55376061435879
what of the an -9.559002325499288
what the ther  -9.65141907898674
what the the t -9.668397794739029
what the the s -9.694297116779557
what the and a -9.702632755988263
what and the a -9.722736598721838
what and the t -9.837373779102077


In [136]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for i in range(10):
  print(bpe.decode(result[i][0]), result[i][1])

gun the and th -0.5170414482503506
gun the the an -2.142614620464978
gun the said t -3.0986610849550758
gun the the th -3.2889598267581444
gun the so the -3.550866545509102
gun the she an -4.013663528410678
gun the saing  -4.332098099824834
gun the the sa -4.470824694857585
gun the was an -4.642094941283447
gun the the so -4.688627932352077
